# Experiment Zero
## Experiment Overview
The purpose of this experiment is to understand the correlation between the primary evaluation metric (ground truth metric) and noise present in data. In order to test this, simulated data is generated and noise is inputted, both in terms of tag positions as well as pose data. The hypothesis is that higher noise should correlate to a higher ground truth metric, and vice versa. For example, inputting noise 